In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/upwork-job-postings-dataset-2024-50k-records/upwork-jobs.csv
/kaggle/input/payload-resume/Aninditaa_Resume_ML.pdf


In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00a 0:00:01


In [4]:
#!pip install --upgrade openai
#!pip install openai==0.28
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 8.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 17.1 MB/s eta 0:00:00


In [5]:

import re
import pandas as pd
import numpy as np
import subprocess
import sys

In [6]:
from PyPDF2 import PdfReader

In [7]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.0 MB/s eta 0:00:0000:01


In [8]:
from sentence_transformers import SentenceTransformer, util

In [9]:
import os

In [ ]:
import openai
openai.api_key = '**************************************************'

In [11]:
# Pandas display settings to prevent truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [12]:
# Dataset Handling - Create a pickle file on first run
csv_file_path = "/kaggle/input/upwork-job-postings-dataset-2024-50k-records/upwork-jobs.csv"  # Replace with your dataset path
pkl_file_path = "job_df.pkl"

if os.path.exists(pkl_file_path):
    # Load from pickle for faster access
    job_df = pd.read_pickle(pkl_file_path)
    print("DataFrame loaded from pickle file.")
else:
    # Read from CSV and save as pickle
    print("Pickle file not found. Loading from CSV and creating pickle.")
    job_df = pd.read_csv(csv_file_path)
    job_df.to_pickle(pkl_file_path)
    print("Pickle file created successfully.")

Pickle file not found. Loading from CSV and creating pickle.
Pickle file created successfully.


In [14]:
# Extract information from the resume using OpenAI GPT
!pip install openai
import openai

def extract_information_from_resume(resume_text):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts structured information from resumes."},
            {"role": "user", "content": f"Extract the following information from the resume below:\n\n1. Skills\n2. Education or Qualifications\n3. Work Experience (including roles and companies)\n4. Total Years of Experience\n\nResume:\n{resume_text}"}
        ],
        max_tokens=1000,
        temperature=0.5
    )

    #extracted_info = response['choices'][0]['message']['content'].strip()
    extracted_info = response.choices[0].message.content
    return extracted_info

extracted_info = extract_information_from_resume(resume_text)
print("Extracted Information from Resume:\n", extracted_info)

Extracted Information from Resume:
 1. Skills:
- Languages: Python, Java, Golang, C/C++
- Tools: GCP, Vertex AI, Kubernetes, Helm, Docker, git
- Frameworks/Technologies: LangChain, Mediapipe, TensorFlow, OpenCV, scikit-learn, Redis, MongoDB

2. Education:
- Master of Science in Computer Science (Thesis), Virginia Tech, Aug 2024 - Present
- Bachelor of Technology in Computer Science and Engineering, Manipal University Jaipur, July 2019 - June 2023

3. Work Experience:
- Software Engineer, Harness [Software Delivery Platform], Bangalore, India, Aug 2023 - July 2024
- Software Engineering Intern, Harness [Software Delivery Platform], Bangalore, India, Feb 2023 - Aug 2023
- Research Intern, CVIT Lab - IIIT Hyderabad, Hyd, India, June 2022 – Sept 2022

4. Total Years of Experience: Based on the provided information, Aninditaa Chauhan has approximately 2 years of work experience.


In [13]:
# Load the resume file
resume_file_path = "/kaggle/input/payload-resume/Aninditaa_Resume_ML.pdf"  # Replace with your file name
if os.path.exists(resume_file_path):
    def extract_text_from_pdf(file_path):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            text = ''.join([page.extract_text() for page in reader.pages])
        return text

    resume_text = extract_text_from_pdf(resume_file_path)
else:
    print(f"Resume file '{resume_file_path}' not found.")
    resume_text = ""

In [15]:
# Parse extracted information
def parse_extracted_info(extracted_text):
    skills_match = re.search(r'Skills:(.*?)(Education|$)', extracted_text, re.DOTALL | re.IGNORECASE)
    education_match = re.search(r'Education:(.*?)(Experience|$)', extracted_text, re.DOTALL | re.IGNORECASE)
    experience_match = re.search(r'Experience:(.*?)(Total Years of Experience|$)', extracted_text, re.DOTALL | re.IGNORECASE)
    years_match = re.search(r'Total Years of Experience:(.*)', extracted_text, re.DOTALL | re.IGNORECASE)

    skills = [skill.strip() for skill in skills_match.group(1).split(',')] if skills_match else []
    education = [degree.strip() for degree in education_match.group(1).split(',')] if education_match else []
    experience = [exp.strip() for exp in experience_match.group(1).split(',')] if experience_match else []
    total_years = years_match.group(1).strip() if years_match else "0"

    return skills, education, experience, total_years

skills, education, experience, total_years = parse_extracted_info(extracted_info)


In [16]:
# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:

# # Function to calculate match score using BERT embeddings and cosine similarity
# def calculate_match_score_bert(job_description, resume_text):
#     job_embedding = model.encode(job_description, convert_to_tensor=True)
#     resume_embedding = model.encode(resume_text, convert_to_tensor=True)
#     similarity_score = util.pytorch_cos_sim(job_embedding, resume_embedding).item()
#     return similarity_score


In [18]:
# Inefficient for large datasets
# if 'description' in job_df.columns:
#     job_df['Match Score'] = job_df['description'].apply(lambda x: calculate_match_score_bert(x, resume_text))

# Optimized batch processing for BERT embeddings
job_descriptions = job_df['description'].tolist()
resume_embedding = model.encode(resume_text, convert_to_tensor=True)

print("Generating job embeddings in batch...")
job_embeddings = model.encode(job_descriptions, convert_to_tensor=True)

# Calculate cosine similarity for all job descriptions at once
print("Calculating cosine similarity...")
similarity_scores = util.pytorch_cos_sim(job_embeddings, resume_embedding).squeeze()

# Assign similarity scores to 'Match Score'
job_df['Match Score'] = similarity_scores.cpu().numpy()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating job embeddings in batch...


Batches:   0%|          | 0/1659 [00:00<?, ?it/s]

Calculating cosine similarity...


In [19]:

# Filtering and displaying top matching jobs (configurable number of top matches)
top_n = 10  # Adjust as needed
matching_jobs = job_df.sort_values(by='Match Score', ascending=False).head(top_n)
print("Top Matching Job Postings:\n", matching_jobs)


Top Matching Job Postings:
                                                                                  title  \
19986       Deep Learning based System for\nECG Classification Using Fourier Transform   
51876            MMDetection Framework Specialist for Custom Faster-RCNN Loss Function   
24274      Machine Learning Expert Needed for Morphological Classification of Galaxies   
9429                                     Medical Imaging AI and automatic segmentation   
24049                                                            Deep learning project   
4295                                                           AI medical data project   
51399                            AI Specialist for Health Prediction Model Development   
27562  Deep Learning Model for Real-Time Object Detection and Current State Prediction   
7955                                         Dynamic K-Selection for Similarity Models   
13006                                                            Senior 

In [26]:
  print("\nTop 10 matching Jobs:")
  print(matching_jobs[['title', 'published_date', 'country', 'Match Score']].head(10))
  


Top 10 matching Jobs:
                                                                                 title  \
19986       Deep Learning based System for\nECG Classification Using Fourier Transform   
51876            MMDetection Framework Specialist for Custom Faster-RCNN Loss Function   
24274      Machine Learning Expert Needed for Morphological Classification of Galaxies   
9429                                     Medical Imaging AI and automatic segmentation   
24049                                                            Deep learning project   
4295                                                           AI medical data project   
51399                            AI Specialist for Health Prediction Model Development   
27562  Deep Learning Model for Real-Time Object Detection and Current State Prediction   
7955                                         Dynamic K-Selection for Similarity Models   
13006                                                            Senior Data 